In [ ]:
# @title Script
if 'loginhug' not in globals():
    !pip install huggingface-sb3
    from huggingface_hub import login
    from huggingface_hub import HfApi
    Custom_1 = "hf_WorGiwfbyFCqKT"
    Custom_2 = "fwwKjTXYBhDjLWFgtYKE"
    Custom_Hg_Token = f"{Custom_1}{Custom_2}"
    read_token = Custom_Hg_Token
    login(read_token, add_to_git_credential=True)
    api = HfApi()
    user = api.whoami(read_token)

import requests, copy, os, torch, gc
from torch import nn
from pathlib import Path
import torch.amp.autocast_mode
from PIL import Image
from unittest.mock import patch
from IPython.display import clear_output,display, HTML
from itertools import islice
import numpy as np
from transformers.dynamic_module_utils import get_imports
from openai import OpenAI
from google import genai
import io, base64, json, yaml
import subprocess, threading, socket, time

gpu_name = torch.cuda.get_device_name()
print(gpu_name)
if 'A100' in gpu_name:
  os.environ['TORCH_CUDA_ARCH_LIST'] = '8.0'
if 'L4' in gpu_name:
  os.environ['TORCH_CUDA_ARCH_LIST'] = '8.9'
if 'T4' in gpu_name:
  os.environ['TORCH_CUDA_ARCH_LIST'] = '7.5'

CLIP_PATH = "google/siglip-so400m-patch14-384"
CHECKPOINT_PATH = Path("/content/joy-caption-alpha-two/cgrkzexw-599808")
TITLE = "<h1><center>JoyCaption Alpha Two (2024-09-26a)</center></h1>"
CAPTION_TYPE_MAP = {
	"Descriptive": [
		"Write a descriptive caption for this image in a formal tone.",
		"Write a descriptive caption for this image in a formal tone within {word_count} words.",
		"Write a {length} descriptive caption for this image in a formal tone.",
	],
	"Descriptive (Informal)": [
		"Write a descriptive caption for this image in a casual tone.",
		"Write a descriptive caption for this image in a casual tone within {word_count} words.",
		"Write a {length} descriptive caption for this image in a casual tone.",
	],
	"Training Prompt": [
		"Write a stable diffusion prompt for this image.",
		"Write a stable diffusion prompt for this image within {word_count} words.",
		"Write a {length} stable diffusion prompt for this image.",
	],
	"MidJourney": [
		"Write a MidJourney prompt for this image.",
		"Write a MidJourney prompt for this image within {word_count} words.",
		"Write a {length} MidJourney prompt for this image.",
	],
	"Booru tag list": [
		"Write a list of Booru tags for this image.",
		"Write a list of Booru tags for this image within {word_count} words.",
		"Write a {length} list of Booru tags for this image.",
	],
	"Booru-like tag list": [
		"Write a list of Booru-like tags for this image.",
		"Write a list of Booru-like tags for this image within {word_count} words.",
		"Write a {length} list of Booru-like tags for this image.",
	],
	"Art Critic": [
		"Analyze this image like an art critic would with information about its composition, style, symbolism, the use of color, light, any artistic movement it might belong to, etc.",
		"Analyze this image like an art critic would with information about its composition, style, symbolism, the use of color, light, any artistic movement it might belong to, etc. Keep it within {word_count} words.",
		"Analyze this image like an art critic would with information about its composition, style, symbolism, the use of color, light, any artistic movement it might belong to, etc. Keep it {length}.",
	],
	"Product Listing": [
		"Write a caption for this image as though it were a product listing.",
		"Write a caption for this image as though it were a product listing. Keep it under {word_count} words.",
		"Write a {length} caption for this image as though it were a product listing.",
	],
	"Social Media Post": [
		"Write a caption for this image as if it were being used for a social media post.",
		"Write a caption for this image as if it were being used for a social media post. Limit the caption to {word_count} words.",
		"Write a {length} caption for this image as if it were being used for a social media post.",
	],
}

HF_TOKEN = os.environ.get("HF_TOKEN", None)

class ImageAdapter(nn.Module):
	def __init__(self, input_features: int, output_features: int, ln1: bool, pos_emb: bool, num_image_tokens: int, deep_extract: bool):
		super().__init__()
		self.deep_extract = deep_extract

		if self.deep_extract:
			input_features = input_features * 5

		self.linear1 = nn.Linear(input_features, output_features)
		self.activation = nn.GELU()
		self.linear2 = nn.Linear(output_features, output_features)
		self.ln1 = nn.Identity() if not ln1 else nn.LayerNorm(input_features)
		self.pos_emb = None if not pos_emb else nn.Parameter(torch.zeros(num_image_tokens, input_features))

		# Other tokens (<|image_start|>, <|image_end|>, <|eot_id|>)
		self.other_tokens = nn.Embedding(3, output_features)
		self.other_tokens.weight.data.normal_(mean=0.0, std=0.02)   # Matches HF's implementation of llama3

	def forward(self, vision_outputs: torch.Tensor):
		if self.deep_extract:
			x = torch.concat((
				vision_outputs[-2],
				vision_outputs[3],
				vision_outputs[7],
				vision_outputs[13],
				vision_outputs[20],
			), dim=-1)
			assert len(x.shape) == 3, f"Expected 3, got {len(x.shape)}"  # batch, tokens, features
			assert x.shape[-1] == vision_outputs[-2].shape[-1] * 5, f"Expected {vision_outputs[-2].shape[-1] * 5}, got {x.shape[-1]}"
		else:
			x = vision_outputs[-2]

		x = self.ln1(x)

		if self.pos_emb is not None:
			assert x.shape[-2:] == self.pos_emb.shape, f"Expected {self.pos_emb.shape}, got {x.shape[-2:]}"
			x = x + self.pos_emb

		x = self.linear1(x)
		x = self.activation(x)
		x = self.linear2(x)

		# <|image_start|>, IMAGE, <|image_end|>
		other_tokens = self.other_tokens(torch.tensor([0, 1], device=self.other_tokens.weight.device).expand(x.shape[0], -1))
		assert other_tokens.shape == (x.shape[0], 2, x.shape[2]), f"Expected {(x.shape[0], 2, x.shape[2])}, got {other_tokens.shape}"
		x = torch.cat((other_tokens[:, 0:1], x, other_tokens[:, 1:2]), dim=1)

		return x

	def get_eot_embedding(self):
		return self.other_tokens(torch.tensor([2], device=self.other_tokens.weight.device)).squeeze(0)

def delete_specific_models():
    variable_names = ['clip_model', 'tokenizer', 'text_model', 'image_adapter']
    for var_name in variable_names:
        if var_name in globals():
            del globals()[var_name]
    torch.cuda.empty_cache()
    gc.collect()

model_list = {
    "APIGemini | 2.5 Pro": "gemini-2.5-pro",
    "APIGemini | 2.5 Flash": "gemini-2.5-flash",
    "APIGemini | 2.5 Flash Lite": "gemini-2.5-flash-lite",
    "APIOpenAI | GPT 5": "gpt-5",
    "APIOpenAI | GPT 5-mini": "gpt-5-mini",
    "APIOpenAI | GPT 5-nano": "gpt-5-nano",
}

url = "https://raw.githubusercontent.com/StableDiffusionVN/SDVN-WebUI/refs/heads/main/model_lib.json"
response = requests.get(url)
model_train_list = json.loads(response.text)

lora_train_py = {
    "Flux": "flux_train_network.py",
    "SDXL": "sdxl_train_network.py",
    "SD15": "train_network.py"
}
db_train_py = {
    "Flux": "flux_train.py",
    "SDXL": "sdxl_train.py",
    "SD15": "train_db.py"
}
def encode_image(image):
    with io.BytesIO() as image_buffer:
        image.save(image_buffer, format="PNG")
        image_buffer.seek(0)
        encoded_image = base64.b64encode(image_buffer.read()).decode('utf-8')
    return encoded_image

def api_check():
    api_file = os.path.join(data_dir,"Setting/API_key_for_sdvn_comfy_node.json")
    if os.path.exists(api_file):
        with open(api_file, 'r', encoding='utf-8') as f:
            api_list = json.load(f)
        return api_list
    else:
        return None

def api_caption(image, length:int, APIkey, Caption, prompt):
    if APIkey == "":
        api_list = api_check()
        if api_check() != None:
            if "Gemini" in Caption:
                APIkey =  api_list["Gemini"]
            if "OpenAI" in Caption:
                APIkey =  api_list["OpenAI"]
    model_name = model_list[Caption]
    prompt += f"Picture description, Send the description on demand, limit {length} words, only send me the answer, Always return English. "
    if 'Gemini' in Caption:
        client = genai.Client(api_key=APIkey)
        response = client.models.generate_content(
                    model=model_name,
                    contents=[prompt, image])
        answer = response.text
    if "OpenAI" in Caption:
        answer = ""
        client = OpenAI(api_key=APIkey)
        image = encode_image(image)
        prompt = [{"type": "input_text", "text": prompt, },
                  {"type": "input_image", "image_url": f"data:image/jpeg;base64,{image}"},]
        messages = [{"role": "user", "content": prompt}]
        response = client.responses.create(
            model=model_name,
            input=messages
        )
        answer = response.output_text
    return answer.strip()

@torch.no_grad()
def joy_caption(input_image: Image.Image, caption_type: str, caption_length: str | int, extra_options: list[str], name_input: str, custom_prompt: str) -> tuple[str, str]:
    torch.cuda.empty_cache()
    # 'any' means no length specified
    length = None if caption_length == "any" else caption_length

    if isinstance(length, str):
        try:
            length = int(length)
        except ValueError:
            pass

    # Build prompt
    if length is None:
        map_idx = 0
    elif isinstance(length, int):
        map_idx = 1
    elif isinstance(length, str):
        map_idx = 2
    else:
        raise ValueError(f"Invalid caption length: {length}")

    prompt_str = CAPTION_TYPE_MAP[caption_type][map_idx]

    # Add extra options
    if len(extra_options) > 0:
        prompt_str += " " + " ".join(extra_options)

    # Add name, length, word_count
    prompt_str = prompt_str.format(name=name_input, length=caption_length, word_count=caption_length)

    if custom_prompt.strip() != "":
        prompt_str = custom_prompt.strip()

    # Preprocess image
    # NOTE: I found the default processor for so400M to have worse results than just using PIL directly
    #image = clip_processor(images=input_image, return_tensors='pt').pixel_values
    image = input_image.resize((384, 384), Image.LANCZOS)
    pixel_values = TVF.pil_to_tensor(image).unsqueeze(0) / 255.0
    pixel_values = TVF.normalize(pixel_values, [0.5], [0.5])
    pixel_values = pixel_values.to('cuda')

    # Embed image
    # This results in Batch x Image Tokens x Features
    with torch.amp.autocast_mode.autocast('cuda', enabled=True):
        vision_outputs = clip_model(pixel_values=pixel_values, output_hidden_states=True)
        embedded_images = image_adapter(vision_outputs.hidden_states)
        embedded_images = embedded_images.to('cuda')

    # Build the conversation
    convo = [
        {
            "role": "system",
            "content": "You are a helpful image captioner.",
        },
        {
            "role": "user",
            "content": prompt_str,
        },
    ]

    # Format the conversation
    convo_string = tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = True)
    assert isinstance(convo_string, str)

    # Tokenize the conversation
    # prompt_str is tokenized separately so we can do the calculations below
    convo_tokens = tokenizer.encode(convo_string, return_tensors="pt", add_special_tokens=False, truncation=False)
    prompt_tokens = tokenizer.encode(prompt_str, return_tensors="pt", add_special_tokens=False, truncation=False)
    assert isinstance(convo_tokens, torch.Tensor) and isinstance(prompt_tokens, torch.Tensor)
    convo_tokens = convo_tokens.squeeze(0)   # Squeeze just to make the following easier
    prompt_tokens = prompt_tokens.squeeze(0)

    # Calculate where to inject the image
    eot_id_indices = (convo_tokens == tokenizer.convert_tokens_to_ids("<|eot_id|>")).nonzero(as_tuple=True)[0].tolist()
    assert len(eot_id_indices) == 2, f"Expected 2 <|eot_id|> tokens, got {len(eot_id_indices)}"

    preamble_len = eot_id_indices[1] - prompt_tokens.shape[0]   # Number of tokens before the prompt

    # Embed the tokens
    convo_embeds = text_model.model.embed_tokens(convo_tokens.unsqueeze(0).to('cuda'))

    # Construct the input
    input_embeds = torch.cat([
        convo_embeds[:, :preamble_len],   # Part before the prompt
        embedded_images.to(dtype=convo_embeds.dtype),   # Image
        convo_embeds[:, preamble_len:],   # The prompt and anything after it
    ], dim=1).to('cuda')

    input_ids = torch.cat([
        convo_tokens[:preamble_len].unsqueeze(0),
        torch.zeros((1, embedded_images.shape[1]), dtype=torch.long),   # Dummy tokens for the image (TODO: Should probably use a special token here so as not to confuse any generation algorithms that might be inspecting the input)
        convo_tokens[preamble_len:].unsqueeze(0),
    ], dim=1).to('cuda')
    attention_mask = torch.ones_like(input_ids)

    generate_ids = text_model.generate(input_ids, inputs_embeds=input_embeds, attention_mask=attention_mask, max_new_tokens=300, do_sample=True, suppress_tokens=None)   # Uses the default which is temp=0.6, top_p=0.9

    # Trim off the prompt
    generate_ids = generate_ids[:, input_ids.shape[1]:]
    if generate_ids[0][-1] == tokenizer.eos_token_id or generate_ids[0][-1] == tokenizer.convert_tokens_to_ids("<|eot_id|>"):
        generate_ids = generate_ids[:, :-1]

    caption = tokenizer.batch_decode(generate_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]

    return caption.strip().replace("\n", "")

def clean_directory(directory):
  supported_types = [".png", ".jpg", ".jpeg", ".webp", ".bmp", ".safetensors"]
  for item in os.listdir(directory):
      file_path = os.path.join(directory, item)
      if os.path.isfile(file_path):
          file_ext = os.path.splitext(item)[1]
          if file_ext not in supported_types:
              print(f"Deleting file {item} from {directory}")
              os.remove(file_path)
      elif os.path.isdir(file_path):
          clean_directory(file_path)

#Florence

version = "large"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float3

def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
    """Workaround for FlashAttention"""
    if os.path.basename(filename) != "modeling_florence2.py":
        return get_imports(filename)
    imports = get_imports(filename)
    return imports

def load_model(version, device):
    from transformers import AutoProcessor, AutoModelForCausalLM
    model_dir = "/content/Model"
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    identifier = "microsoft/Florence-2-" + version

    with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports):
        model = AutoModelForCausalLM.from_pretrained(identifier, torch_dtype=torch_dtype, cache_dir=model_dir, trust_remote_code=True).to(device)
        processor = AutoProcessor.from_pretrained(identifier, cache_dir=model_dir, trust_remote_code=True)

    model = model.to(device)
    return (model, processor)

def load(version, device):
  if 'processor' not in globals():
    global model, processor
    model, processor = load_model(version, device)

def run_example(task_prompt, image, max_new_tokens, num_beams, do_sample, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=max_new_tokens,
        early_stopping=False,
        do_sample=do_sample,
        num_beams=num_beams,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

def florence_caption(task_prompt, image, max_new_tokens = 1024, num_beams = 3, do_sample = False, fill_mask = False, text_input=None):
    result = run_example(task_prompt, image, max_new_tokens, num_beams, do_sample)
    return result[task_prompt].replace("\n", "")

#Florence

def caption_dir(image_dir,prompt):
  if Caption == 'Florence':
    load(version, device)
  for img_file in os.listdir(image_dir):
      file_path = os.path.join(image_dir, img_file)
      if os.path.isdir(file_path) :
          caption_dir(file_path,prompt)
      if img_file.lower().endswith((".png", ".jpg", ".jpeg", ".webp", ".bmp", ".PNG", ".JPG", ".JPEG")):
          img_path = os.path.join(image_dir, img_file)
          image = Image.open(img_path).convert("RGB")
          if Caption == 'Florence':
            cap = florence_caption(prompt,image).replace('The image shows','')
          elif Caption == 'Joy_Caption':
            cap = joy_caption(image, Joy_Type, Cap_prompt[Caption_Length][3], [Joy_Extra_Option], Joy_Character_Name, Joy_Custom_Prompt)
          else:
            cap = api_caption(image, Cap_prompt[Caption_Length][3], APIkey, Caption, API_Prompt)
          txt_path = os.path.join(image_dir, f"{os.path.splitext(img_file)[0]}{extension}")
          with open(txt_path, "w") as f:
              f.write(cap)
          print(f"Miêu tả của ảnh {img_file}: {cap}")
  torch.cuda.empty_cache()
def read_file(filename):
    with open(filename, "r") as f:
        contents = f.read()
    return contents

def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)

def process_tags(filename, custom_tag, append, remove_tag):
    contents = read_file(filename)
    if remove_tag:
      contents = contents.replace(custom_tag, "")
    else:
      tags = [tag.strip() for tag in contents.split(',')]
      custom_tags = [tag.strip() for tag in custom_tag.split(',')]
      for custom_tag in custom_tags:
          custom_tag = custom_tag.replace("_", " ")
          if custom_tag not in tags:
              if append:
                  tags.append(custom_tag)
              else:
                  tags.insert(0, custom_tag)
      contents = ', '.join(tags)
    write_file(filename, contents)

def check_dir(image_dir):
  if not any([filename.endswith(extension) for filename in os.listdir(image_dir)]):
      for filename in os.listdir(image_dir):
          if filename.endswith(((".png", ".jpg", ".jpeg", ".webp", ".bmp", ".PNG", ".JPG", ".JPEG"))):
              open(
                  os.path.join(image_dir, filename.split(".")[0] + extension),
                  "w",
              ).close()

def process_dir(image_dir, tag, append, remove_tag):
  check_dir(image_dir)
  for filename in os.listdir(image_dir):
      file_path = os.path.join(image_dir, filename)
      if os.path.isdir(file_path) :
          print(filename)
          process_dir(file_path, tag, append, remove_tag)
      elif filename.endswith(extension):
          process_tags(file_path, tag, append, remove_tag)

def add_forder_name(folder):
  for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    if os.path.isdir(file_path):
      folder_name = os.path.basename(file_path)
      try:
          steps, name = folder_name.split('_', 1)
          steps = int(steps)
      except ValueError:
          name = folder_name
      name = name.replace("/", ", ")
      process_dir(file_path, name, False, False)
      add_forder_name(file_path)

def get_steps(folder):
    folder_name = os.path.basename(folder)
    try:
        steps, name = folder_name.split('_', 1)
        steps = int(steps)
    except ValueError:
        steps = Steps
        name = folder_name
    return steps, name

def random_sample(folder, control_folder):
  import random
  file_list = get_supported_images(control_folder)
  control_path = random.choice(file_list)
  data_path = control_path.replace(control_folder,folder)
  txt_path = data_path.replace(data_path.split(".")[-1], "txt")
  try:
    sample = read_file(txt_path)
    sample = sample.replace('"', r'\"')
  except IndexError:
    sample = "girl portrait, smile"
  return [sample, control_path]

def get_supported_images(folder):
  import glob
  supported_extensions = (".png", ".jpg", ".jpeg", ".webp", ".bmp", ".PNG", ".JPG", ".JPEG")
  list_img = [file for ext in supported_extensions for file in glob.glob(f"{folder}/*{ext}")]
  for img_file in os.listdir(folder):
      file_path = os.path.join(folder, img_file)
      if os.path.isdir(file_path) :
          list_img = list_img + get_supported_images(file_path)
  return list_img

def replace(old_string, new_string):
    import re
    with open(file_path, 'r') as file:
        yaml_content = file.read()
    updated_content = re.sub(old_string, new_string, yaml_content)
    with open(file_path, 'w') as file:
        file.write(updated_content)

default_value = {
    "config_dir" : "/content/SDVN-training-colab-flux/Config",
    "Lora_name" : "lora_name",
    "folder_train": "/content/drive/MyDrive/SD-Data",
    "OutputFolder": "/content/drive/MyDrive/SD-Data/Lora",
    "Custom_Caption": "",
    "Dim": 32,
    "Alpha": 16,
    "Save_steps": 1000,
    "Resolution": "1024",
    "Batch_size": 4,
    "Train_TexEncoder": False,
    "Lr": 1e-4,
    "Sampler_Steps": 100,
    "Low_VRAM": False,
    "Sampler_Prompt": "",
}
def check_value(default_value):
    for key, value in default_value.items():
        if key not in globals():
            globals()[key] = value
        elif type(globals()[key]) == str:
            globals()[key] = globals()[key].split(' ')[-1]

type_config = {
    "Flux": ["black-forest-labs/FLUX.1-dev","flux"],
    "Flux Kontext": ["black-forest-labs/FLUX.1-Kontext-dev","flux_kontext"],
    "Qwen": ["Qwen/Qwen-Image","qwen_image"],
    "Qwen-Edit": ["Qwen/Qwen-Image-Edit","qwen_image_edit"],
    "Hi-Dream": ["HiDream-ai/HiDream-I1-Full","hidream"],
    "Hi-Dream E1": ["HiDream-ai/HiDream-E1-1","hidream_e1"],
    "Wan2.2 TI2V 5B": ["Wan-AI/Wan2.2-TI2V-5B-Diffusers","wan22_5b"],
    "Wan2.1 I2V (14B-720P)": ["Wan-AI/Wan2.1-I2V-14B-720P-Diffusers", "wan21_i2v:14b"],
    "Wan2.1 I2V (14B-480P)": ["Wan-AI/Wan2.1-I2V-14B-480P-Diffusers","wan21_i2v:14b480p"],
    "Wan2.1 (14B)": ["Wan-AI/Wan2.1-T2V-14B-Diffusers","wan21:14b"],
    "Wan2.2 (14B)": ["ai-toolkit/Wan2.2-T2V-A14B-Diffusers-bf16","wan22_14b:t2v"],
    "Wan2.2 I2V (14B)": ["ai-toolkit/Wan2.2-I2V-A14B-Diffusers-bf16","wan22_14b_i2v"],
}

def config(folder):
  sample_prompt_ = Sampler_Prompt
  sample_image_path_ = Control_Img_Path
  check_value(default_value)
  with open(f'{config_dir}/config.yaml', 'r') as file:
    config_dict = yaml.safe_load(file)
  file_path = f'{config_dir}/config_{folder["name"]}.yaml'

  config_dict["config"]["name"] = Lora_name if len(folder_train) == 1 else folder["name"]

  config_dict["config"]["process"][0]["training_folder"] = OutputFolder
  config_dict["config"]["process"][0]["trigger_word"] = Custom_Caption if Custom_Caption != "" else folder["name"]

  config_dict["config"]["process"][0]["network"]["linear"] = Dim
  config_dict["config"]["process"][0]["network"]["linear_alpha"] = Alpha
  config_dict["config"]["process"][0]["save"]["save_every"] = Save_steps

  config_dict["config"]["process"][0]["datasets"][0]["folder_path"] = folder["path"]
  config_dict["config"]["process"][0]["datasets"][0]["control_path"] = folder["control_path"] if ControlFolder != "" else None
  config_dict["config"]["process"][0]["datasets"][0]["resolution"] = [int(x) for x in Resolution.split(',')]

  config_dict["config"]["process"][0]["train"]["batch_size"] = Batch_size
  config_dict["config"]["process"][0]["train"]["steps"] = folder["steps"]
  config_dict["config"]["process"][0]["train"]["train_text_encoder"] = Train_TexEncoder
  config_dict["config"]["process"][0]["train"]["lr"] = Lr
  config_dict["config"]["process"][0]["train"]["disable_sampling"] = False if Sampler_Steps > 0 else True
  
  config_dict["config"]["process"][0]["model"]["name_or_path"] = type_config[TypeTrain][0]
  config_dict["config"]["process"][0]["model"]["arch"] = type_config[TypeTrain][1]
  config_dict["config"]["process"][0]["model"]["low_vram"] = Low_VRAM

  if folder["control_path"] != "":
      s, p = random_sample(folder["path"],folder["control_path"])
  else:
      s, p = random_sample(folder["path"],folder["path"])
  if sample_prompt_ == "" :
      sample_prompt_ = s
  if sample_image_path_ == "":
      sample_image_path_ = p
  if TypeTrain in ['Flux Kontext', 'Qwen-Edit', 'Hi-Dream E1', "Wan2.1 I2V (14B-480P)", "Wan2.1 I2V (14B-720P)", "Wan2.2 I2V (14B)"]:
    sample_prompt_ = f"{sample_prompt_} --ctrl_img {sample_image_path_}"
    config_dict["config"]["process"][0]["sample"]["width"] = 1024 if sample_image_path_ == "" else image_size(sample_image_path_)[0]
    config_dict["config"]["process"][0]["sample"]["height"] = 1024 if sample_image_path_ == "" else image_size(sample_image_path_)[1]
  print(sample_prompt_)
  config_dict["config"]["process"][0]["sample"]["sample_every"] = Sampler_Steps if Sampler_Steps > 0 else 100
  config_dict["config"]["process"][0]["sample"]["samples"][0]["prompt"] = sample_prompt_

  config_dict["meta"]["name"] = Lora_name if len(folder_train) == 1 else folder["name"]
  config_dict["meta"]["version"] = "Train by trainlora.vn | stablediffusion.vn"
  with open(file_path, 'w') as file:
      yaml.dump(config_dict, file, default_flow_style=False)

  print('=====================')
  print(f'Thư mục train: {folder["path"]}')
  print(f'  Số lượng ảnh: {len(get_supported_images(folder["path"]))}')
  print(f'  Số Steps: {folder["steps"]}')
  print(f'  Output: {OutputFolder}/{folder["name"]}')
  print('=====================')

def image_size(image_path, max_size=1536):
    with Image.open(image_path) as img:
        width, height = img.size
    if max(width, height) <= max_size:
        return width, height
    scale_ratio = max_size / max(width, height)
    new_width = int(width * scale_ratio)
    new_height = int(height * scale_ratio)
    return new_width, new_height

def train():
  %cd {toolkit_dir}
  delete_specific_models()
  for folder in folder_train:
    !python run.py {config_dir}/config_{folder["name"]}.yaml

def cloudflare_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('172.28.0.12', port))
      if result == 0:
        break
      sock.close()
  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://172.28.0.12:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print(f"\033[92m{'🔗 Link online để sử dụng (delay 1 phút):'}\033[0m", l[l.find("http"):], end='')

def sever_flare(port):
  threading.Thread(target=cloudflare_thread, daemon=True, args=(port,)).start()

clear_output()